In [91]:
import numpy as np # 
import pandas as pd # 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/zennureozbekacar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [92]:
# load data
df = pd.read_json('resources/News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment.""",2018-05-26


In [93]:
df = df.drop_duplicates(subset='short_description', keep='last')

In [94]:
df=df.reset_index()

In [95]:
import re
import string

def clean_text_round(text):
    
    text = text.lower()
    text = re.sub('\[.*?\]', '',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[''""...]','',text)
    text = re.sub('\n','',text)
    text = re.sub('\d+', '', text)
    text = re.sub('[^\w\s]','', text)  # remove punctuation 
    
    return text

get_cleaning = lambda x: clean_text_round(x)
df.short_description = pd.DataFrame(df.short_description.apply(get_cleaning))
df.short_description = df.short_description.str.replace('\d+', '') # remove numbers

In [96]:
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [97]:
df['short_description'] = df['short_description'].apply(lambda x: gensim.parsing.preprocessing.remove_stopwords(x))


In [139]:
df.short_description[2]

'actor longtime girlfriend anna eberstein tied knot civil ceremony'

In [140]:
remove_words = ['i', 'your', 'you', 'on', 'with', 'and', 'have', 'the', 'to', 'in', 'for', 'that', 'had', 'be', 'a', 'year'
               'it', 'may', 'one', 'as', 'if', 'is', 'via', 'this', 'will', 'david', 'jenni', 'im', 'susan', 'it', 'up',
               'angelica', 'hi', 'hello', 'we', 'our', 'all', 'kelli', 'yes', 'gavin', 'our', 'were', 'of', 'can', 'at',
               'any', 'by', 'also', 'joe', 'ronnie', 'morning', 'evening', 'good', 'what', 'okay', 'ok', 'are', 'us', 'my',
               'th', 'st', 'nd', 'rd', 'was', 'there', 'then', 'lee', 'out', 'or', 'so','alan', 'from','unfortunately',
               'alason', 'but', 'youre', 'does', 'heres', 'little', 'more', 'set', 'br', 'dr', 'ave', 'here', 'about', 'an',
               'let', 'know', 'than', 'then', 'no', 'why', 'way', 'every', 'thats', 'taken', 'today', 'way', 'id', 'isnt', 
               'only', 'bay', 'me', 'when', 'want', 'end', 'month', 'do', 'til', 'get', 'back', 'thanks', 'bonnie', 'woodal',
               'off', 'drive', 'michell','and', 'he','she', 'her','just']

In [141]:
def apply_remove_words_to_texts():
    pat = r'\b(?:{})\b'.format('|'.join(remove_words))
    df["short_description_new"] = df['short_description'].str.replace(pat, '')
    df["short_description_new"] = df['short_description_new'].str.strip()
    return df

In [142]:
df = apply_remove_words_to_texts()

In [143]:
df.head()

,index,category,headline,authors,link,short_description,date,label,short_description_new
0,0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,left husband killed children day america,2018-05-26,Negative,left husband killed children day america
1,1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,course song,2018-05-26,Neutral,course song
2,2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,actor longtime girlfriend anna eberstein tied knot civil ceremony,2018-05-26,Neutral,actor longtime girlfriend anna eberstein tied knot civil ceremony
3,3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,actor gives dems asskicking fighting hard donald trump,2018-05-26,Negative,actor gives dems asskicking fighting hard donald trump
4,4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,dietland actress said bags cathartic therapeutic moment,2018-05-26,Neutral,dietland actress said bags cathartic therapeutic moment


In [144]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [155]:
analyzer = SentimentIntensityAnalyzer()
overall_sentiment = []

for art in df['short_description_new']:
    sentiment_dict=analyzer.polarity_scores(art)
    #print("Overall sentiment dictionary is : ", sentiment_dict) 
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
    #print("Sentence Overall Rated As", end = " ")  
    if sentiment_dict['compound'] >= 0.05 : 
        overall_sentiment.append("Positive")
    elif sentiment_dict['compound'] <= - 0.05 : 
        overall_sentiment.append("Negative")
    else : 
        overall_sentiment.append("Neutral")
        
df['label'] = overall_sentiment

In [156]:
df.head()

,index,category,headline,authors,link,short_description,date,label,short_description_new
0,0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,left husband killed children day america,2018-05-26,Negative,left husband killed children day america
1,1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,course song,2018-05-26,Neutral,course song
2,2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,actor longtime girlfriend anna eberstein tied knot civil ceremony,2018-05-26,Neutral,actor longtime girlfriend anna eberstein tied knot civil ceremony
3,3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,actor gives dems asskicking fighting hard donald trump,2018-05-26,Negative,actor gives dems asskicking fighting hard donald trump
4,4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,dietland actress said bags cathartic therapeutic moment,2018-05-26,Neutral,dietland actress said bags cathartic therapeutic moment


In [157]:
df['category'].value_counts()

POLITICS          29106
WELLNESS          17549
ENTERTAINMENT     13251
TRAVEL            9375 
STYLE & BEAUTY    9152 
PARENTING         8624 
FOOD & DRINK      6206 
QUEER VOICES      5405 
HEALTHY LIVING    5197 
BUSINESS          5060 
COMEDY            4361 
SPORTS            4186 
BLACK VOICES      4064 
HOME & LIVING     4023 
THE WORLDPOST     3657 
WEDDINGS          3645 
PARENTS           3433 
DIVORCE           3401 
IMPACT            3050 
WOMEN             3023 
CRIME             2671 
MEDIA             2259 
WORLD NEWS        2169 
WEIRD NEWS        2152 
GREEN             2034 
TECH              1990 
TASTE             1904 
RELIGION          1779 
SCIENCE           1763 
MONEY             1706 
STYLE             1500 
ARTS & CULTURE    1321 
ENVIRONMENT       1317 
WORLDPOST         1242 
FIFTY             1042 
GOOD NEWS         1027 
LATINO VOICES     1018 
CULTURE & ARTS    1016 
COLLEGE           921  
EDUCATION         891  
ARTS              863  
Name: category, 

News API categories
* business
* entertainment
* general
* health
* science
* sports
* technology

In [158]:
business =['BUSINESS','MONEY']
entertainment=['ENTERTAINMENT','ARTS','COMEDY','ARTS & CULTURE','CULTURE & ARTS','MEDIA']
health=['HEALTH','WELLNESS','HEALTHY LIVING']
science=['SCIENCE']
sports=['SPORTS']
technology = ['tech']

In [159]:
condition = df['category'].isin(business)
businessDF = df[condition]
businessDF.shape

(6766, 9)

In [160]:
from sklearn.model_selection import train_test_split

trainData, testData = train_test_split(businessDF, test_size=0.3)

In [161]:
len(businessDF)

6766

In [162]:
len(trainData)

4736

In [163]:
len(testData)

2030

In [164]:
trainData.head()

,index,category,headline,authors,link,short_description,date,label,short_description_new
91473,99910,BUSINESS,"Women in Business Q&A: Jan Jones Blackhurst, EVP of Communications, Gov Relations and Corporate Responsibility, Caesars Entertainment","Laura Dunn, ContributorSocial Media and Communications Professional, Founder and Edit...",https://www.huffingtonpost.com/entry/women-in-business-qa-jan_b_6588414.html,jan jones blackhurst executive vice president communications government relations corporate responsibility caesars entertainment capacity oversees worldwide government affairs corporate communications community relations corporatesocial responsibility programs entire company,2015-02-01,Positive,jan jones blackhurst executive vice president communications government relations corporate responsibility caesars entertainment capacity oversees worldwide government affairs corporate communications community relations corporatesocial responsibility programs entire company
173161,195580,BUSINESS,Rich Won't Move Even If State Raises Income Taxes: Report,Bonnie Kavoussi,https://www.huffingtonpost.com/entry/taxing-the-rich_us_5bb2baeee4b0480ca65ac0d6,maybe states need rush fast everfamous race state tax increases little,2012-03-25,Neutral,maybe states need rush fast everfamous race state tax increases
77581,80175,BUSINESS,Men Educated At Ivy League Schools Vastly Outearn Female Classmates,Emily Peck,https://www.huffingtonpost.com/entry/ivy-league-income-gap-by-gender_us_55f860c0e4b00e2cd5e82ae2,lot choice major oh sexism,2015-09-15,Neutral,lot choice major oh sexism
94556,105765,BUSINESS,Economic History and Thanksgiving,"Kevin Price, ContributorPublisher and Editor in Chief, US Daily Review",https://www.huffingtonpost.com/entry/economic-history-and-than_b_6209386.html,thanksgiving certainly favorite holidays adult children christmas lost charm things enjoyable getting fat watching holiday specials especially important football games,2014-11-25,Positive,thanksgiving certainly favorite holidays adult children christmas lost charm things enjoyable getting fat watching holiday specials especially important football games
8552,8657,BUSINESS,"Apple Apologizes For Slowed iPhones, Drops Price Of Battery Replacements",Carla Herreria,https://www.huffingtonpost.com/entry/apple-apologizes-for-slowing-iphones_us_5a457c1de4b06d1621b7dea9,company dropped battery prices customers arent satisfied,2017-12-29,Negative,company dropped battery prices customers arent satisfied


In [165]:
trainData.to_csv("Resources/BusinessTrainData_1.csv")
testData.to_csv("Resources/BusinessTestData_1.csv")

In [166]:
trainData.sample(frac=1).head(5) # shuffle the df and pick first 5

,index,category,headline,authors,link,short_description,date,label,short_description_new
87907,93509,BUSINESS,Congress Is Facing Its Last Call to Stand Up for Fair Trade,"James P. Hoffa, ContributorGeneral President, International Brotherhood of Teamsters",https://www.huffingtonpost.com/entry/congress-is-facing-its-la_b_7042722.html,complicated trade agreements thousands workers unemployment line hamper nations economy shouldnt elected officials chance review changes lawmakers certainly spent significant time considering frivolous matters recent years,2015-04-16,Positive,complicated trade agreements thousands workers unemployment line hamper nations economy shouldnt elected officials chance review changes lawmakers certainly spent significant time considering frivolous matters recent years
86593,91286,BUSINESS,5 Ways to be a More Positive Leader,"Michelle McQuaid, Contributorbest-selling author, workplace wellbeing teacher and playful c...",https://www.huffingtonpost.com/entry/5-ways-to-be-a-more-posit_b_7254402.html,ceo recently asked wheres roadmap positive leader great question present researchers exploring exactly road map look like,2015-05-11,Positive,ceo recently asked wheres roadmap positive leader great question present researchers exploring exactly road map look like
151642,173671,MONEY,Best Cash-Back Credit Cards for Holiday Shopping,"Jeff Hindenach, Contributor\nDirector of Content, NextAdvisor.com",https://www.huffingtonpost.com/entry/best-cashback-credit-card_us_5b9c8798e4b03a1dcc7f7e51,cashback credit cards effective holiday season putting bigticket items cashback cards reap nice rewards choose cards carefully cashback credit cards created equally,2012-11-16,Positive,cashback credit cards effective holiday season putting bigticket items cashback cards reap nice rewards choose cards carefully cashback credit cards created equally
87693,93135,BUSINESS,Does Meerkating Count as Networking?,"Vivy Chao, ContributorEducator & Entrepreneur",https://www.huffingtonpost.com/entry/does-meerkating-count-as-_b_7082896.html,thought act networking actually unnatural think person needs attend event knowing attendees act like normal walk complete strangers shake hands begin conversing,2015-04-20,Positive,thought act networking actually unnatural think person needs attend event knowing attendees act like normal walk complete strangers shake hands begin conversing
82496,85169,BUSINESS,"At Zappos, Pushing Shoes and a Vision",,https://www.huffingtonpost.comhttp://www.nytimes.com/2015/07/19/business/at-zappos-selling-shoes-and-a-vision.html,sizzling june morning las vegas zapposcom employees sat airconditioned conference room decorated,2015-07-20,Neutral,sizzling june las vegas zapposcom employees sat airconditioned conference room decorated


# Vectorizing the Data

In [167]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['short_description_new'])
test_vectors = vectorizer.transform(testData['short_description_new'])

In [168]:
train_vectors

<4736x2519 sparse matrix of type '<class 'numpy.float64'>'
	with 41255 stored elements in Compressed Sparse Row format>

In [169]:
test_vectors

<2030x2519 sparse matrix of type '<class 'numpy.float64'>'
	with 16896 stored elements in Compressed Sparse Row format>

# Creating a Linear SVM Mode

In [170]:
import time
# from sklearn import svm
from sklearn.svm import SVC 
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config

In [171]:
# Perform classification with SVM, kernel=linear
classifier = SVC(kernel='linear')
t0 = time.time()
t1 = time.time()
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
set_config(display="diagram")
pipe = Pipeline([('classifier', classifier)])
pipe

Pipeline(steps=[('classifier', SVC(kernel='linear'))])

In [172]:
# classifier.fit(train_vectors, trainData['label'])
pipe.fit(train_vectors, trainData['label'])

Pipeline(steps=[('classifier', SVC(kernel='linear'))])

In [173]:
predicted=pipe.predict(test_vectors)

In [174]:
svm_acc_train=pipe.score(train_vectors, trainData['label'])
svm_acc_test=pipe.score(test_vectors, testData['label'])

In [175]:
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['label'], predicted, output_dict=True)
print(f"Train Accuracy = {svm_acc_train}")
print(f"Test Accuracy = {svm_acc_test}")
print('positive: ', report['Positive'])
print('negative: ', report['Negative'])

Training time: 0.000040s; Prediction time: 0.000048s
Train Accuracy = 0.8811233108108109
Test Accuracy = 0.6458128078817734
positive:  {'precision': 0.7136237256719185, 'recall': 0.7762096774193549, 'f1-score': 0.7436021245774987, 'support': 992}
negative:  {'precision': 0.6108870967741935, 'recall': 0.5519125683060109, 'f1-score': 0.5799043062200956, 'support': 549}


In [176]:
from sklearn.metrics import classification_report
print(classification_report(pipe.predict(test_vectors),testData['label']))

              precision    recall  f1-score   support

    Negative       0.55      0.61      0.58       496
     Neutral       0.49      0.52      0.50       455
    Positive       0.78      0.71      0.74      1079

    accuracy                           0.65      2030
   macro avg       0.60      0.62      0.61      2030
weighted avg       0.66      0.65      0.65      2030



# Save the Model

In [ ]:
# import pickle
# # pickling the vectorizer
# pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
# # pickling the model
# pickle.dump(classifier_linear, open('classifier.sav', 'wb'))

# Test the SVM classifier

In [ ]:
# review = """SUPERB, I AM IN LOVE IN THIS PHONE"""
# review_vector = vectorizer.transform([review]) # vectorizing
# print(classifier_linear.predict(review_vector))

In [78]:
# f = open('Resources/BusinessTrainData_1.csv', 'r', encoding='utf8')
# text = f.read()
# t_list = text.split('\n')

# corpus = []

# for sentence in t_list:
#     corpus.append(sentence.split())


In [79]:
# print(corpus[:10])

In [80]:
# vector = Word2Vec(corpus, size=100, min_count=1,sg=1)

In [81]:
# vector.wv['app', 'council']

In [82]:
# vector.wv.most_similar('council')

In [83]:
# vector.wv.most_similar('app')

In [84]:
# vector.save('word2vec.vector')

In [85]:
# vector = Word2Vec.load('word2vec.vector')

In [86]:
# def closestwords_tsneplot(model, word):
#     word_vectors = np.empty((0,100))
#     word_labels = [word]
    
#     close_words = model.wv.most_similar(word)
    
#     word_vectors = np.append(word_vectors, np.array([model.wv[word]]), axis=0)
    
#     for w, _ in close_words:
#         word_labels.append(w)
#         word_vectors = np.append(word_vectors, np.array([model.wv[w]]), axis=0)
        
#     tsne = TSNE(random_state=0)
#     Y = tsne.fit_transform(word_vectors)
    
#     x_coords = Y[:, 0]
#     y_coords = Y[:, 1]
    
#     plt.scatter(x_coords, y_coords)
    
#     for label, x, y in zip(word_labels, x_coords, y_coords):
#         plt.annotate(label, xy=(x, y), xytext=(5, -2), textcoords='offset points')
        
#     plt.show()

In [87]:
# closestwords_tsneplot(model, 'council')

In [88]:
# closestwords_tsneplot(model, 'firearms')

In [89]:
# closestwords_tsneplot(model, 'Bezos')

In [90]:
# closestwords_tsneplot(model, 'scandal')